In [1]:
import requests
from bs4 import BeautifulSoup

page = 1
# 거래글
page_url = "https://search.naver.com/search.naver"
params = {
    'where': "articlec",
    "query": "에어팟",
    "strat": (page - 1) * 10 + 1,
}
res = requests.get(page_url, params=params)
res

<Response [200]>

In [2]:
html = res.text
soup = BeautifulSoup(html, "html.parser")

In [3]:
tag_list = soup.select('.sh_cafe_top')
len(tag_list)

10

In [4]:
tag = tag_list[0]

In [5]:
title_tag = tag.select_one('.sh_cafe_title')
post_url = title_tag["href"]
post_title = title_tag.text
post_desc = tag.select_one('.sh_cafe_passage').text

price = tag.select_one(".cafe_item_price .num").text

print(post_title, post_url)
print(post_desc)
print(price)

에어팟 2세대 유선충전 모델 풀박스 https://cafe.naver.com/appleiphone/5306327
개인거래(판매), 판매완료, 가격 140,000원, 직접거래, 에어팟 2세대 유선충전 모델 풀박스, 애플 에어팟 2세대 유선 충전 모델입니다 구성품은 풀박스 케이...
140,000


In [6]:
if tag.select_one(".cafe_item_price del"):
    price = "판매 완료"
else:
    price = tag.select_one(".cafe_item_price .num").text

In [7]:
#is_sell = tag.select_one(".cafe_item_price del") is None
#is_sell # 판매 완료 여부

In [8]:
# 또는 다음과 같이 판매완료 여부 확인 가능
is_sell = "done" in tag.select_one('.cafe_item_status')['class']
is_sell

True

In [17]:
# 카페명, url, date, img
cafe_tag = tag.select_one('.txt_block a')
cafe_title = cafe_tag.text
cafe_url = cafe_tag['href']
print(cafe_title, cafe_url)

아사모 -[아이폰, 아이폰 Xr, Xs... https://cafe.naver.com/appleiphone


In [19]:
post_date = tag.select_one('.txt_inline').text

In [25]:
img_url = tag.select_one(".thumb-rollover img")["src"]
img_url

'https://search.pstatic.net/common/?src=http%3A%2F%2Fcafefiles.naver.net%2FMjAxOTEwMDVfMTA4%2FMDAxNTcwMjEzMjg5NzEx.KjRPwa4ANjSWjGJZGICLYVlpILK27kx3NCC0CpYyba0g.fKPj7jUCp8ZFappf8YW7hrUwrITmlCjRQT7Xhx4-IIwg.JPEG%2FexternalFile.jpg&type=m80_80#960x720'

In [33]:
# 항목 합치기

post_list = []

for tag in tag_list: 
    title_tag = tag.select_one('.sh_cafe_title')
    cafe_tag = tag.select_one('.txt_block a')
    
    post_list.append(dict(
        post_url = title_tag["href"],
        post_title = title_tag.text,
        post_desc = tag.select_one('.sh_cafe_passage').text,
        price = tag.select_one(".cafe_item_price .num").text,
        is_sell = 'sell' in tag.select_one('.cafe_item_status')['class'],
        cafe_title = cafe_tag.text,
        cafe_url = cafe_tag['href'],
        post_date = tag.select_one('.txt_inline').text,
        img_url = tag.select_one(".thumb img")["src"],
    ))

In [34]:
post_list

[{'post_url': 'https://cafe.naver.com/appleiphone/5306327',
  'post_title': '에어팟 2세대 유선충전 모델 풀박스',
  'post_desc': '개인거래(판매), 판매완료, 가격 140,000원, 직접거래, 에어팟 2세대 유선충전 모델 풀박스, 애플 에어팟 2세대 유선 충전 모델입니다 구성품은 풀박스 케이...',
  'price': '140,000',
  'is_sell': False,
  'cafe_title': '아사모 -[아이폰, 아이폰 Xr, Xs...',
  'cafe_url': 'https://cafe.naver.com/appleiphone',
  'post_date': '2019.10.05. ',
  'img_url': 'https://search.pstatic.net/common/?src=http%3A%2F%2Fcafefiles.naver.net%2FMjAxOTEwMDVfMTA4%2FMDAxNTcwMjEzMjg5NzEx.KjRPwa4ANjSWjGJZGICLYVlpILK27kx3NCC0CpYyba0g.fKPj7jUCp8ZFappf8YW7hrUwrITmlCjRQT7Xhx4-IIwg.JPEG%2FexternalFile.jpg&type=m80_80#960x720'},
 {'post_url': 'https://cafe.naver.com/fx8300/527442',
  'post_title': '미개봉 에어팟 2 유선충전모델',
  'post_desc': '개인거래(판매), 판매완료, 가격 145,000원, 직접거래, 미개봉 에어팟 2 유선충전모델, 미개봉 에어팟 2세대 유선충전 모델 애플 정품입니다 초기불량 발생시...',
  'price': '145,000',
  'is_sell': False,
  'cafe_title': 'AMD 비쉐라 라이젠 컴퓨터 사용자...',
  'cafe_url': 'https://cafe.naver.com/fx8300',
  'post_date': '19시간 전

In [39]:
for post in filter(lambda post: post['is_sell'], post_list):  # is_sell 중인 것만 추출
    print('{post_date} {post_title} {price}원 at {cafe_title}'.format(**post))

2일 전  에어팟2세대 무선충전 135,000원 at [거사모] 거제도 대표 여행,펜션...
2019.09.24.  에어팟 1세대 팜 100,000원 at 맥 쓰는 사람들 [구 맥북을 쓰는...


In [45]:
mobile_page_url = "https://m.cafe.naver.com/inmacbook/2138764"
res = requests.get(mobile_page_url)
res

<Response [200]>

In [46]:
"카페 멤버만" in res.text

True

In [47]:
# 검색페이지를 거쳐서 가게 되면 보여주게 되어 있음

from urllib.parse import quote

검색어 = "에어팟"
quote(검색어)

'%EC%97%90%EC%96%B4%ED%8C%9F'

In [48]:
# post page 접근하기
mobile_page_url = "https://m.cafe.naver.com/inmacbook/2138764"

headers = {
    'Referer': "https://m.cafe.naver.com/SectionMarketArticleSearch.nhn?query="+quote(검색어)
}

res = requests.get(mobile_page_url, headers=headers)
res

<Response [200]>

In [49]:
"카페 회원만" in res.text

False